spaceship titanic with keras and deep learning 
Application to the kaggle competition by Imanol Elizondo Perucich.

In [1]:
#import libraries
import tensorflow as tf 
import numpy as np
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing

In [2]:
#read the dataset with pandas
data=pd.read_csv("C:\\Users\\elizo\\Desktop\\proyectos\\spacetitan\\train.csv") #path to files
test=pd.read_csv("C:\\Users\\elizo\\Desktop\\proyectos\\spacetitan\\test.csv") 
testid=test["PassengerId"] #save this column


In [3]:
#eliminate the name column and fill in the missing values
def clean(data):
    data=data.drop(["Name"], axis=1)
    cols=["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","Age"]
    for col in cols:
        data[col].fillna(data[col].median(),inplace=True)
    data.HomePlanet.fillna("uknown", inplace=True)
    data.CryoSleep.fillna(False, inplace=True)
    data.Destination.fillna("uknown", inplace=True)
    data.VIP.fillna(False, inplace=True)
    data.Cabin.fillna("u/0/u", inplace=True)

    return data



In [4]:
# The cabin column has 3 pieces of data: deck, number and side, so we created an individual column for each.
def cabin(dat):
    a=[]
    b=[]
    c=[]
    for i in (range(len(dat["Cabin"]))):
        a.append(dat["Cabin"][i].split("/")[0])
        b.append(dat["Cabin"][i].split("/")[1])
        c.append(dat["Cabin"][i].split("/")[2])
    dat["cabin0"]=a
    dat["cabin1"]=b
    dat["cabin2"]=c
    return dat
    


In [5]:
#The second part of the passengerid column gives us the number of people within a group so we extract the ones that are together in another column
def pasid(dat):
    x=[]
    for i in range (len(dat["PassengerId"])):
       if(i+2>len(dat["PassengerId"])):
           x.append(0)
       else:
            if dat["PassengerId"][i].split("_")[1]=="01" and dat["PassengerId"][i+1].split("_")[1]=="01" :
                x.append(0)
            else:
                x.append(1)
    
    dat["family"]=x
    return dat

In [6]:
#now we extract the number of people in each grup
def group(dat):
    x=[]
    for i in range (len(dat["PassengerId"])):
       x.append(int(dat["PassengerId"][i].split("_")[0]))
    
    dat["groupn"]=x
    return dat

In [7]:
#we run the functions for the test and data 
data=clean(data)
test=clean(test)
data=cabin(data)
test=cabin(test)
data=pasid(data)
test=pasid(test)
data=group(data)
test=group(test)
#we delete the extra columns 
data=data.drop(["PassengerId"], axis=1)
test=test.drop(["PassengerId"], axis=1)
data=data.drop(["Cabin"], axis=1)
test=test.drop(["Cabin"], axis=1)




C:\Users\elizo\AppData\Local\Temp\ipykernel_11676\1126586702.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(),inplace=True)
C:\Users\elizo\AppData\Local\Temp\ipykernel_11676\1126586702.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [8]:
#We transform text data into numbers
le=preprocessing.LabelEncoder()
cols=["HomePlanet","CryoSleep","Destination","VIP","cabin0","cabin2"]
for col in cols:
    data[col]=le.fit_transform(data[col])
    test[col]=le.transform(test[col])
    
data.head()
    

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,cabin0,cabin1,cabin2,family,groupn
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0,0,0,1
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,5,0,1,0,2
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,0,1,1,3
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,0,1,1,3
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,5,1,1,0,4


In [9]:
data["Transported"]=le.fit_transform(data["Transported"])

In [10]:
#we select our y and x and transform them into numpy arrays 
y =data ["Transported"]
x= data.drop("Transported",axis=1)
x = np.asarray(x).astype(np.float32)
y = np.asarray(y).astype(np.float32)
 

In [11]:
x.shape

(8693, 15)

In [12]:
#we build our model with layers
model=tf.keras.Sequential([
    keras.layers.InputLayer(input_shape=(15)),
    keras.layers.BatchNormalization(input_shape=(x.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),  # Dropout regularization
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.3),  # Dropout regularization
    
    keras.layers.Dense(1, activation='sigmoid')
    
    
    
    ])
#we compile our model 
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 15)                60        
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 64)                1024      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                        

In [14]:
#we train our model with a 20% validation split and 200 epochs
model.fit(x,y,validation_split=.2, epochs=200 ,batch_size=32, verbose=1)

Epoch 1/200
218/218 [==============================] - 2s 5ms/step - loss: 0.6086 - accuracy: 0.6616 - val_loss: 0.4892 - val_accuracy: 0.7867
Epoch 2/200
218/218 [==============================] - 1s 3ms/step - loss: 0.5214 - accuracy: 0.7476 - val_loss: 0.4896 - val_accuracy: 0.7826
Epoch 3/200
218/218 [==============================] - 1s 3ms/step - loss: 0.5139 - accuracy: 0.7587 - val_loss: 0.4693 - val_accuracy: 0.7970
Epoch 4/200
218/218 [==============================] - 1s 3ms/step - loss: 0.4948 - accuracy: 0.7637 - val_loss: 0.4860 - val_accuracy: 0.7752
Epoch 5/200
218/218 [==============================] - 1s 3ms/step - loss: 0.4898 - accuracy: 0.7714 - val_loss: 0.5032 - val_accuracy: 0.7504
Epoch 6/200
218/218 [==============================] - 1s 3ms/step - loss: 0.4794 - accuracy: 0.7728 - val_loss: 0.4965 - val_accuracy: 0.7573
Epoch 7/200
218/218 [==============================] - 1s 3ms/step - loss: 0.4755 - accuracy: 0.7778 - val_loss: 0.4820 - val_accuracy: 0.7683

In [15]:
#transform the test data into a numpy array and predict the results 
test = np.asarray(test).astype(np.float32)
pred=model.predict(test)
binary_predictions = (pred > 0.5).astype(int).flatten()


134/134 [==============================] - 0s 2ms/step


In [16]:
#make a pandas dataframes with the results 
df=pd.DataFrame({"PassengerId":testid.values, "Transported" : binary_predictions})

In [17]:
#Our data is in numeric values ​​so we transform it into boolean values.
for i in range(len(df["Transported"])):
    if (df["Transported"][i]==0):
        df["Transported"][i]=False
    else:
        df["Transported"][i]=True

C:\Users\elizo\AppData\Local\Temp\ipykernel_11676\1288858976.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Transported"][i]=True
C:\Users\elizo\AppData\Local\Temp\ipykernel_11676\1288858976.py:5: SettingWithCopyWarning: 
A value is

In [18]:
print(df)

     PassengerId Transported
0        0013_01        True
1        0018_01       False
2        0019_01        True
3        0021_01        True
4        0023_01       False
...          ...         ...
4272     9266_02        True
4273     9269_01       False
4274     9271_01        True
4275     9273_01       False
4276     9277_01        True

[4277 rows x 2 columns]


In [19]:
#we make a csv file with the final results 
df.to_csv("submisions.csv",index=False)